In [1]:
#set the working directory
import os
os.chdir("/media/dftortosa/Windows/Users/dftor/Documents/diego_docs/industry/data_incubator/capstone_project/")

#check
os.getcwd()

'/media/dftortosa/Windows/Users/dftor/Documents/diego_docs/industry/data_incubator/capstone_project'

In [2]:
#VADER can be used from vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#open instance
analyzer = SentimentIntensityAnalyzer()

In [3]:
import re
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()).replace("#", "")

##CHECK REGULAR EXPRESSIONS
#https://www.sciencedirect.com/science/article/pii/S104244312030072X

In [4]:
#define function to be applied per row
def sentiment_per_row(raw_tweet, sentiment_type="compound"):
        
    #clean
    tweet_cleaned = clean_tweet(raw_tweet)
        #import preprocessor as p; p.clean()
    
    #get the negative sentiment
    sentiment = analyzer.polarity_scores(tweet_cleaned)[sentiment_type]

    #return
    return sentiment

In [5]:
###CHECK THIS!!!

#define a function to do a given operation grouping by day but considering the a previous date
def groupby_calcs(operation, metric_name, args="", n_previous_days=2, currency=""):
        
    #get the string with the operation name
    string_evaluate = 'sentiment_results_df' + currency + '.groupby("date")["sentiment"].' + operation + '(' + args + ').to_frame()'
        #do operations for the sentiment of each day and then convert to df

    #evaluate to get the DF with the calculations per day
    data_frame = eval(string_evaluate)
    
    #extract the range of days for which we have data
    time_range = pd.date_range(np.min(np.unique(data_frame.index)), 
                        np.max(np.unique(data_frame.index)))
    
    #fill with NaN those dates with data
    data_frame = data_frame.reindex(time_range, fill_value=np.NaN)
        #https://stackoverflow.com/questions/15610805/accessing-row-from-previous-day-in-pandas-dataframe-with-apply
    
    #get sentiment of the previous days
    data_frame['sentiment'] = data_frame['sentiment'].shift(n_previous_days)
        #https://stackoverflow.com/questions/19324453/add-missing-dates-to-pandas-dataframe
        
    #reset the index
    data_frame = data_frame.reset_index(level=0)
        #set the index (date) as column 
  
    #change the column name for sentiment
    data_frame = data_frame.rename(columns={'index': 'date', 'sentiment': metric_name + '_sent' + currency + '_previous_' + str(n_previous_days) + '_days'})

    #append the df to the list
    return data_frame
    

def groupby_calcs_across_dates(previous_date, currency=""):
    
    #define a list of operations to do
    metrics_sentiment = ["quantile_0.05",
                         "quantile_0.1",
                         "quantile_0.2",
                         "quantile_0.3",
                         "quantile_0.4",
                         "quantile_0.5",
                         "quantile_0.6",
                         "quantile_0.7",
                         "quantile_0.8",
                         "quantile_0.9",
                         "quantile_0.95",
                         "mean",
                         "min",
                         "max",
                         "sum",
                         "count", 
                             #this can be useful to control for averages with low number of tweets
                             #but also to consider how much people is talking about europes economy 
                         "range", 
                         "var", 
                         "std", 
                         "iqr"]
    
    #open list to save each calculation
    list_data_frames = []
    
    #apply each operation
    for metric in metrics_sentiment:
        
        #for std
        if metric == "std":
            #use ddof=0 to get zero for cases with 1 tweet per day (see above)
            result_calc = groupby_calcs(operation=metric, 
                          metric_name=metric, 
                          args='ddof=0', 
                          n_previous_days=previous_date, 
                          currency=currency)
        elif "quantile" in metric: #if quantile
            #get the operation name and the quantile number
            operation_name = metric.split("_")[0]
            quantile_number = metric.split("_")[1]
            result_calc = groupby_calcs(operation=operation_name, 
                          metric_name=metric, 
                          args=quantile_number, 
                          n_previous_days=previous_date, 
                          currency=currency) 
        elif "range" in metric:
            result_calc = groupby_calcs(operation="apply", 
                          metric_name=metric, 
                          args="lambda x: np.max(x) - np.min(x)",
                          n_previous_days=previous_date, 
                          currency=currency)
        elif "iqr" in metric:
            result_calc = groupby_calcs(operation="apply", 
                          metric_name=metric, 
                          args="lambda x: (np.percentile(x, 75) - np.percentile(x, 25))",
                          n_previous_days=previous_date, 
                          currency=currency)   
        else: 
            result_calc = groupby_calcs(operation=metric, 
                          metric_name=metric, 
                          n_previous_days=previous_date, 
                          currency=currency)
           
        list_data_frames.append(result_calc)
        
    return list_data_frames

In [ ]:
import pandas as pd
import numpy as np

tweets_df_usd = pd.read_json("data/json_files/hashtag_fed_twitter_1999_1_1.json", lines=True)

#reset the indeces to avoid problems when indexing
tweets_df_usd = tweets_df_usd.reset_index(drop=True)
    #use drop drop parameter to avoid the old index being added as a column

#check for duplicated IDs
tweets_df_usd["id"].duplicated().any()

#take a look to one of the duplicated IDs
#example_duplicated_id = tweets_df_usd[tweets_df_usd["id"].duplicated()].iloc[2,:].loc["id"] #get the ID

#get the URL for all tweets having the example ID and check if there are identical URLs
#tweets_df.loc[tweets_df["id"] == example_duplicated_id, "url"].duplicated().any()
    #tweets with the same ID have the same URL, so they are the same tweet
    
#remove tweets with a duplicated id
tweets_df_usd = tweets_df_usd.drop_duplicates(subset=['id'])

#select only tweets in english
tweets_df_usd_en = tweets_df_usd[tweets_df_usd["lang"] == "en"]
print(tweets_df_usd_en.shape)
    #CHECK IF THE SENTIMENT MODEL USED CAN DEAL WITH OTHER LANGUAGES
    
    
del(tweets_df_usd)

import multiprocessing as mp

with mp.Pool(10) as pool:
    tweets_df_usd_en["sentiment"] = pool.map(sentiment_per_row, tweets_df_usd_en["rawContent"])
        #for each row in tweets_df_usd_en["rawContent"], apply sentiment_per_row across 10 cores
        #https://stackoverflow.com/questions/45545110/make-pandas-dataframe-apply-use-all-cores
    pool.close()
    
#CHECK: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

#change the sign so positive sentiment is negative for EUR
tweets_df_usd_en["sentiment"] = -tweets_df_usd_en["sentiment"]


#get the date and sentiment
sentiment_results_df_usd = tweets_df_usd_en[["date", "sentiment"]]
sentiment_results_df_usd

del(tweets_df_usd_en)

#leave only year, month and day
sentiment_results_df_usd["date"] = pd.to_datetime(sentiment_results_df_usd["date"].dt.strftime('%Y-%m-%d'))
    #https://stackoverflow.com/questions/38067704/how-to-change-the-datetime-format-in-pandas
    #https://stackoverflow.com/questions/20689288/converting-pandas-columns-to-datetime64-including-missing-values
sentiment_results_df_usd

#CHECK WARNING

#check we have rows ordered in chronologicla order
sentiment_results_df_usd.equals(sentiment_results_df_usd.sort_index())

#get sentiment 1 and 2 weeks ago, along with 1, 2, 3, 4 and 5 months ago
list_dates = np.arange(1, 16, 1)
#list_dates = [7, 14, 30, 60, 90, 120, 150, 180, 210, 240, 270]

#open a pool with 10 processes
pool_dates = mp.Pool(processes=10)

#apply the function for each previous day in parallel across 10 cores
list_data_frames_usd = [pool_dates.apply_async(groupby_calcs_across_dates, args=(previous_date, "_usd")).get() for previous_date in list_dates]
    #https://stackoverflow.com/questions/42843203/how-to-get-the-result-of-multiprocessing-pool-apply-async

#close the pool
pool_dates.close()

#flatten list
flat_list_usd = [item for sublist in list_data_frames_usd for item in sublist]
flat_list_usd

In [ ]:
##merge all data.frames
from functools import reduce

#merge them using reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['date'], how='inner'), [element for element in flat_list_usd])
    #leave only those rows having date in the datasets of currency value and sentiment
    #https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
df_merged
    #note that here the previous eur pricing can be different than the eur pricing of the previous row
    #because when merging we lose days without tweet sentiment, so there is no longer continuous days

In [ ]:
#remove these NA cases
df_merged = df_merged.dropna()
df_merged

In [ ]:
#save the final data.frame with all metrics and eur pricing
df_merged.to_csv("results/twitter_sentiment_data_usd.csv.gz", compression="gzip", index=False)
#df_merged = pd.read_csv("results/twitter_sentiment_data.csv.gz", compression="gzip")